In [233]:
import torch
import torch.nn.functional as F 
import matplotlib.pyplot as plt 
%matplotlib inline

In [234]:
words=open("names.txt",'r').read().splitlines()

In [235]:
chars=sorted(list(set(''.join(words))))   
stoi={s: i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [236]:
block_size=3
x,y=[],[]
for w in words:
    context=[0]*block_size
    for ch in w+'.':
        ix=stoi[ch]
        x.append(context)
        y.append(ix)
        #print(''.join(itos[i] for i in context),'---->',itos[ix])
        context=context[1:]+[ix]

x=torch.tensor(x)
y=torch.tensor(y)    


In [250]:
def create_dataset(words):
    block_size=3
    x,y=[],[]
    for w in words:
        context=[0]*block_size
        for ch in w+'.':
            ix=stoi[ch]
            x.append(context)
            y.append(ix)
            #print(''.join(itos[i] for i in context),'---->',itos[ix])
            context=context[1:]+[ix]

    x=torch.tensor(x)
    y=torch.tensor(y)

    return x,y  

import random 
random.seed(32)
random.shuffle(words)
n1=int(0.8*len(words))
n2=int(0.9*len(words))

xtr,ytr=create_dataset(words[:n1])
xdev,ydev=create_dataset(words[n1:n2])
xte,yte=create_dataset(words[n2:])

28829

In [238]:
x.shape,x.dtype,y.shape,y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [239]:
c=torch.randn((27,2))
w1=torch.randn((6,100))
b1=torch.randn((100))
w2=torch.randn((100,27))
b2=torch.randn((27))
para=[c,w1,b2,w2,b2]

In [240]:
for i in para:
    i.requires_grad=True

In [241]:
sum([i.nelement() for i in para])
    

3408

In [324]:
for i in range(10000):
    ix=torch.randint(0,xtr.shape[0],(32,))
    emb=c[xtr[ix]]
    h=torch.tanh(emb.view(-1,6)@w1+b1)
    logits=h@w2+b2
    loss=F.cross_entropy(logits,ytr[ix])
    for p in para:
        p.grad=None
    loss.backward()
    
    for i in para:
        i.data+=-0.01*i.grad
print(loss)

tensor(2.1124, grad_fn=<NllLossBackward0>)


In [325]:
emb=c[xtr]
h=torch.tanh(emb.view(-1,6)@w1+b1)
logits=h@w2+b2
loss=F.cross_entropy(logits,ytr)
loss

tensor(2.2592, grad_fn=<NllLossBackward0>)

In [326]:
emb=c[xdev]
h=torch.tanh(emb.view(-1,6)@w1+b1)
logits=h@w2+b2 
loss=F.cross_entropy(logits,ydev)
loss


tensor(2.2726, grad_fn=<NllLossBackward0>)